# Data-Set Overview Plots

In [ ]:
using QAOA, Distributions, Interpolations
using DataFrames, Arrow, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

using PyCall
np = pyimport("numpy")

PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)
Base.show(io::IO, ::MIME"text/latex", df::AbstractDataFrame) = show(df, backend=:html, show_row_number=true, highlighters=:tf_html_default)

normalize!(X) = X ./ maximum(filter(!isnan, X));

## Small-/Large-gap data comparison

In [ ]:
N = 9
# N = 11
# N = 13
# N = 15
# N = 17
# N = 19

patterns_dict = Dict(
    9  => r"random_SK_instance_N_9_seed_(\d+)\.h5",
    11 => r"random_SK_instance_N_11_seed_(\d+)\.h5",
    13 => r"random_SK_instance_N_13_seed_(\d+)\.h5",
    15 => r"random_SK_instance_N_15_seed_(\d+)\.h5",
    17 => r"random_SK_instance_N_17_seed_(\d+)\.h5",
    19 => r"random_SK_instance_N_19_seed_(\d+)\.h5"
);

### Loading

In [ ]:
npts = 2048
# npts = 2^13
coarse_times = range(0, 1, npts + 1)
exact_times = range(0, 1, 33);

In [ ]:
subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)
filter!(x -> !occursin("results", x), instance_names)
filter!(x -> !occursin("undecided", x), instance_names)
filter!(x -> !occursin("frustrated", x), instance_names)
filter!(x -> !occursin("main_df", x), instance_names);

In [ ]:
# most_frustrated_spins_seeds = h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6)))
# most_frustrated_spins_idxs = h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)));
# most_frustrated_spins = Dict(zip(most_frustrated_spins_seeds, eachrow(most_frustrated_spins_idxs)))
# main_df = DataFrame(Arrow.Table(folder_name * "main_df_undecided.arrow"));

In [ ]:
most_frustrated_spins_seeds = h5read(folder_name * @sprintf("most_frustrated_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6)))
most_frustrated_spins_idxs = h5read(folder_name * @sprintf("most_frustrated_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)));
most_frustrated_spins = Dict(zip(most_frustrated_spins_seeds, eachrow(most_frustrated_spins_idxs)))
main_df = DataFrame(Arrow.Table(folder_name * "main_df_frustrated.arrow"));

In [ ]:
names(main_df)

## Plots

### Shifted to center

In [ ]:
padded_exact_times = range(0, 1, 2(size(exact_times)[1] - 1) + 1)
padded_coarse_times = range(0, 1, 2(size(coarse_times)[1] - 1) + 1);

#### Average plots

In [ ]:
function get_mean_shifted_fracs(main_df)
    mean_shifted_fracs = []
    # for row in eachrow(first(main_df, num_instances))
    for row in eachrow(main_df)
        # total_fracs = sum(all_adiabatic_fracs[seed])
        total_fracs = row[:adiabatic_fracs][1]

        # get maximum from adiabatic fraction alongside index and time
        max_frac = maximum(total_fracs)
        max_frac_idx = findfirst(x -> x == max_frac, total_fracs)
        max_frac_time = exact_times[max_frac_idx]
        push!(mean_shifted_fracs, shift_idx_to_center(max_frac_idx, total_fracs))
    end

    mean(mean_shifted_fracs)
end

In [ ]:
function get_mean_shifted_crit_flucs(main_df, most_frustrated_spins)
    
    mean_shifted_crit_flucs = []
    # num_instances = 20
    # for row in eachrow(first(main_df, num_instances))
    for row in eachrow(main_df)
        # print(row[:seed], "\t")
        
        # get maximum from adiabatic fraction alongside index and time
        total_fracs = sum(row[:adiabatic_fracs])
        max_frac = maximum(total_fracs)
        max_frac_idx = findfirst(x -> x == max_frac, total_fracs)
        max_frac_time = exact_times[max_frac_idx]

        # find that also for coarse_times
        max_frac_idx_coarse = findfirst(x -> x == max_frac_time, coarse_times)

        frustrated_flucs_1 = row[:scaled_most_frustrated_flucs][1]
        frustrated_flucs_2 = row[:scaled_most_frustrated_flucs][2]

        top_idx = most_frustrated_spins[row[:seed]][1]
        sec_idx = most_frustrated_spins[row[:seed]][2]
        magnetizations = reshape(row[:magnetizations], (length(most_frustrated_spins[row[:seed]]), npts+1))

        inner_navg = 64
        outer_navg = 64 # 256
        # inv_mag_1 = smoothen(smoothen(shift_idx_to_center(max_frac_idx_coarse, magnetizations[top_idx, :]), padded_coarse_times, navg=inner_navg) .|> abs, padded_coarse_times, navg=outer_navg)
        # inv_mag_2 = smoothen(smoothen(shift_idx_to_center(max_frac_idx_coarse, magnetizations[sec_idx, :]), padded_coarse_times, navg=inner_navg) .|> abs, padded_coarse_times, navg=outer_navg)
        # crit_flucs = [smoothen(shift_idx_to_center(max_frac_idx_coarse, frustrated_flucs_1), padded_coarse_times) ./ inv_mag_1, 
        #               smoothen(shift_idx_to_center(max_frac_idx_coarse, frustrated_flucs_2), padded_coarse_times) ./ inv_mag_2]

        inv_mag_1 = smoothen(smoothen(magnetizations[top_idx, :], coarse_times, navg=inner_navg) .|> abs, coarse_times, navg=outer_navg)
        inv_mag_2 = smoothen(smoothen(magnetizations[sec_idx, :], coarse_times, navg=inner_navg) .|> abs, coarse_times, navg=outer_navg)
        crit_flucs = [shift_idx_to_center(max_frac_idx_coarse, smoothen(frustrated_flucs_1, coarse_times) ./ inv_mag_1), 
                      shift_idx_to_center(max_frac_idx_coarse, smoothen(frustrated_flucs_2, coarse_times) ./ inv_mag_2)]          
        
        # which_spin(X) = abs(0.5 - padded_coarse_times[findfirst(x -> x == maximum(filter(!isnan, X)), X)])
        # crit_idx = sort([(1, which_spin(crit_flucs[1])), (2, which_spin(crit_flucs[2]))], by=x->x[2])[1][1]
        crit_idx = reverse!(sort([(1, normalize!(crit_flucs[1])[npts+1]), (2, normalize!(crit_flucs[2])[npts+1])], by=x->x[2]))[1][1]

        # push!(mean_shifted_crit_flucs, crit_flucs[crit_idx] |> normalize!)
        # push!(mean_shifted_crit_flucs, maximum(total_fracs) .* normalize!(crit_flucs[crit_idx]))
        # push!(mean_shifted_crit_flucs, maximum(total_fracs) .* (crit_flucs[crit_idx]))
        push!(mean_shifted_crit_flucs,  crit_flucs[crit_idx])
    end
    mean_shifted_crit_flucs
end

In [ ]:
N_vals = [9, 11, 13, 15, 17, 19]
all_mean_shifted_fracs = []
all_mean_shifted_crit_flucs = []
for N in N_vals
    try
        folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)

        # most_frustrated_spins_seeds = h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6)))
        # most_frustrated_spins_idxs = h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)));
        # most_frustrated_spins = Dict(zip(most_frustrated_spins_seeds, eachrow(most_frustrated_spins_idxs)))
        # main_df = DataFrame(Arrow.Table(folder_name * "main_df_undecided.arrow"))      
        
        most_frustrated_spins_seeds = h5read(folder_name * @sprintf("most_frustrated_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6)))
        most_frustrated_spins_idxs = h5read(folder_name * @sprintf("most_frustrated_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)));
        most_frustrated_spins = Dict(zip(most_frustrated_spins_seeds, eachrow(most_frustrated_spins_idxs)))
        main_df = DataFrame(Arrow.Table(folder_name * "main_df_frustrated.arrow"))

        mean_shifted_fracs = get_mean_shifted_fracs(main_df)
        mean_shifted_crit_flucs = get_mean_shifted_crit_flucs(main_df, most_frustrated_spins)
        push!(all_mean_shifted_fracs, mean_shifted_fracs)    
        push!(all_mean_shifted_crit_flucs, mean_shifted_crit_flucs)   
    catch err
        println(N, ", ", err, ", ") 
    end
end

In [ ]:
figure(figsize=(12, 4))
for (k, mean_shifted_fracs) in enumerate(all_mean_shifted_fracs)
    ax = subplot(2, 6, k)
    axvline(0, c="k", alpha=0.25, lw=5)
    semilogy(padded_exact_times .- 1/2, mean_shifted_fracs, "-ok", lw=1.5, ms=3)#, label=@sprintf("\$N = %i\$", N_vals[k] - 1))
    semilogy([], [], "-k", lw=0 , label=@sprintf("\$N = %i\$", N_vals[k] - 1))
    xlim(-0.5, 0.5)
    ax.set_xticklabels([-0.25, 0.0, 0.25])
    ylim(1e-1, 1e3)
    ax.set_xticklabels([])
    if k == 1
        ylabel("Average \$ g_1(s)  \$")
    else
        ax.set_yticklabels([])
    end   
    # legend(frameon=false, handlelength=0, markerscale=0, borderaxespad=0.1, fontsize=12, loc="upper right")
    legend(frameon=false, borderaxespad=0.1, fontsize=12, loc="upper right")
end

all_y_data = mean.(all_mean_shifted_crit_flucs)
for (k, y_data) in enumerate(all_y_data)
    println(length(all_mean_shifted_crit_flucs[k]))
    ax = subplot(2, 6, k + 6)
    axvline(0, c="k", alpha=0.25, lw=5)
    plot(padded_coarse_times .- 1/2, normalize!(y_data), "-k")
    xlabel("\$s - s_*\$")
    xlim(-0.5, 0.5)
    # ax.set_xticklabels(["\$-\\frac 12\$", "\$0\$", "\$\\frac 12\$"], fontsize=12)
    ax.set_xticklabels(["\$-1/2\$", "\$0\$", "\$1/2\$"], fontsize=12)
    ylim(0, 1)
    if k == 1
        ylabel("Average \$ \\widetilde{\\chi}_{i_*}(s) \$")
    else
        ax.set_yticklabels([])
    end    
end
tight_layout(h_pad=0.0, w_pad=0.5)
savefig(PLOT_PATH * "disorder_stats.pdf", dpi=256, bbox_inches="tight")

In [ ]:
figure(figsize=(12, 4))
for (k, mean_shifted_fracs) in enumerate(all_mean_shifted_fracs)
    ax = subplot(2, 6, k)
    semilogy(padded_exact_times .- 1/2, mean_shifted_fracs, "-ok", lw=1.5, ms=3, label=@sprintf("\$N = %i\$", N_vals[k] - 1))
    xlim(-0.5, 0.5)
    ax.set_xticklabels([-0.25, 0.0, 0.25])
    ylim(1e-1, 1e3)
    ax.set_xticklabels([])
    if k == 1
        ylabel("Average \$ g_1(s)  \$")
    else
        ax.set_yticklabels([])
    end   
    legend(frameon=false, handlelength=0, markerscale=0, borderaxespad=0.1, loc="upper left")
end
# ylabel("\$ \\left\\langle\\hspace{-2mm}\\left\\langle\\frac{|\\langle 1|H_Z-H_X|0\\rangle|}{E_1 - E_0} \\right\\rangle\\hspace{-2mm}\\right\\rangle\$")
# ylabel("\$ \\left\\langle\\hspace{-0.8mm}\\langle g_1(s) \\right\\rangle\\hspace{-1mm}\\rangle \$")
# ax.set_xticklabels([])

all_y_data = mean.(all_mean_shifted_crit_flucs)
for (k, y_data) in enumerate(all_y_data)
    ax = subplot(2, 6, k + 6)
    axvline(0, c="k", alpha=0.25, lw=5)
    semilogy(padded_coarse_times .- 1/2, y_data, "-k")
    xlabel("\$s - s_*\$")
    xlim(-0.5, 0.5)
    ax.set_xticklabels(["\$-\\frac 12\$", "\$0\$", "\$\\frac 12\$"])
    # ax.set_xticklabels(["\$-1/2\$", "\$0\$", "\$1/2\$"])
    ylim(1e-1, 1e1)
    if k == 1
        ylabel("Average \$ \\chi_{i_*}(s)  \$")
    else
        ax.set_yticklabels([])
    end    
end
# ylim(0, )
# ylabel("\$ \\langle\\delta \\bar z_i(s) \\delta z_i(s) \\rangle \$")
# ylabel("\$ \\left\\langle\\hspace{-0.8mm}\\langle \\chi(s) \\right\\rangle\\hspace{-1mm}\\rangle \$")
tight_layout(pad=0.25)

In [ ]:
mean_shifted_flucs = []
# for row in eachrow(first(main_df, num_instances))
for row in eachrow(main_df)
    # total_fracs = sum(all_adiabatic_fracs[seed])
    total_fracs = row[:adiabatic_fracs][1]

    # get maximum from adiabatic fraction alongside index and time
    max_frac = maximum(total_fracs)
    max_frac_idx = findfirst(x -> x == max_frac, total_fracs)
    max_frac_time = exact_times[max_frac_idx]

    # find that also for coarse_times
    max_frac_idx_coarse = findfirst(x -> x == max_frac_time, coarse_times)
    push!(mean_shifted_flucs, shift_idx_to_center(max_frac_idx_coarse, row[:scaled_most_frustrated_flucs][1]))
end

mean_shifted_flucs = mean(mean_shifted_flucs);

In [ ]:
figure(figsize=(4, 4))
ax = subplot(211)
# plot(padded_exact_times, mean_shifted_fracs, "-ok", lw=1.5, ms=3)
semilogy(padded_exact_times .- 1/2, mean_shifted_fracs, "-ok", lw=1.5, ms=3)
xlim(-0.5, 0.5)
ylim(1e-1, 1e2)
ylabel("\$\\frac{|\\langle 1|H_Z-H_X|0\\rangle|}{E_1 - E_0}\$")
ax.set_xticklabels([])

ax = subplot(212)
y_data = smoothen(mean_shifted_flucs, padded_coarse_times, navg=128)
# y_data = mean_shifted_flucs
plot(padded_coarse_times .- 1/2, y_data ./ maximum(y_data))
xlim(-0.5, 0.5)
ylim(0, 1)
xlabel("\$s - s_*\$")
ylabel("\$ \\langle\\delta \\bar z_i(s) \\delta z_i(s) \\rangle \$")
tight_layout()

#### Single-instance plots

In [ ]:
num_instances = 20

for row in eachrow(first(main_df, num_instances))
    print(row[:seed], "\t")
    # frac_1, frac_2, frac_3, frac_4 = row[:adiabatic_fracs]
    # frac_1, frac_2 = all_adiabatic_fracs[plot_seed]
    total_fracs = sum(row[:adiabatic_fracs])

    # get maximum from adiabatic fraction alongside index and time
    max_frac = maximum(total_fracs)
    max_frac_idx = findfirst(x -> x == max_frac, total_fracs)
    max_frac_time = exact_times[max_frac_idx]

    # find that also for coarse_times
    max_frac_idx_coarse = findfirst(x -> x == max_frac_time, coarse_times)

    frustrated_flucs_1 = row[:scaled_most_frustrated_flucs][1]
    frustrated_flucs_2 = row[:scaled_most_frustrated_flucs][2]

    figure(figsize=(4, 3))
    title(row[:seed])
    ax = subplot(211)
    plot(padded_exact_times, shift_idx_to_center(max_frac_idx, row[:adiabatic_fracs][1]), lw=2.5)
    plot(padded_exact_times, shift_idx_to_center(max_frac_idx, row[:adiabatic_fracs][2]), lw=2.5)
    plot(padded_exact_times, shift_idx_to_center(max_frac_idx, total_fracs), "-k", lw=2.5)
    xlim(0., 1.)
    ylim(0, 10)
    ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
    ax.set_xticklabels([])
    # legend(frameon=false)

    ax = subplot(212)
    # plot(padded_coarse_times,  shift_idx_to_center(max_frac_idx_coarse, frustrated_flucs_1 .- row[:mean_scaled_flucs])  |> normalize!, label=row[:seed])
    # plot(padded_coarse_times,  shift_idx_to_center(max_frac_idx_coarse, frustrated_flucs_1) |> normalize!)

    top_idx = most_frustrated_spins[row[:seed]][1]
    sec_idx = most_frustrated_spins[row[:seed]][2]
    magnetizations = reshape(row[:magnetizations], (N-1, npts+1))

    inner_navg = 64
    outer_navg = 64
    # inv_mag_1 = smoothen(smoothen(shift_idx_to_center(max_frac_idx_coarse, magnetizations[top_idx, :]), padded_coarse_times, navg=inner_navg) .|> abs, padded_coarse_times, navg=outer_navg)
    # inv_mag_2 = smoothen(smoothen(shift_idx_to_center(max_frac_idx_coarse, magnetizations[sec_idx, :]), padded_coarse_times, navg=inner_navg) .|> abs, padded_coarse_times, navg=outer_navg)
    # crit_flucs = [smoothen(shift_idx_to_center(max_frac_idx_coarse, frustrated_flucs_1), padded_coarse_times) ./ inv_mag_1, 
    #               smoothen(shift_idx_to_center(max_frac_idx_coarse, frustrated_flucs_2), padded_coarse_times) ./ inv_mag_2]

    inv_mag_1 = smoothen(smoothen(magnetizations[top_idx, :], coarse_times, navg=inner_navg) .|> abs, coarse_times, navg=outer_navg)
    inv_mag_2 = smoothen(smoothen(magnetizations[sec_idx, :], coarse_times, navg=inner_navg) .|> abs, coarse_times, navg=outer_navg)
    crit_flucs = [shift_idx_to_center(max_frac_idx_coarse, smoothen(frustrated_flucs_1, coarse_times) ./ inv_mag_1), 
                  shift_idx_to_center(max_frac_idx_coarse, smoothen(frustrated_flucs_2, coarse_times) ./ inv_mag_2)]                  
    
    which_spin(X) = abs(0.5 - padded_coarse_times[findfirst(x -> x == maximum(filter(!isnan, X)), X)])
    crit_idx = sort([(1, which_spin(crit_flucs[1])), (2, which_spin(crit_flucs[2]))], by=x->x[2])[1][1]
    # print(crit_idx, "\t")
    crit_idx = reverse!(sort([(1, normalize!(crit_flucs[1])[npts+1]), (2, normalize!(crit_flucs[2])[npts+1])], by=x->x[2]))[1][1]
    # println(crit_idx)
    # println(maximum(filter(!isnan, crit_flucs[1])), ", ", maximum(filter(!isnan, crit_flucs[2])), ", ", shift_idx_to_center(max_frac_idx, total_fracs) |> maximum)
    spin_max_loc(X) = padded_coarse_times[findfirst(x -> x == maximum(filter(!isnan, X)), X)]
    axvline.([spin_max_loc(crit_flucs[1]), spin_max_loc(crit_flucs[2])], c="k")

    # plot(padded_coarse_times, crit_flucs[crit_idx] |> normalize!, "-k", label=row[:seed])
    # plot(padded_coarse_times, crit_flucs[1] |> normalize!, "--C0")
    # plot(padded_coarse_times, crit_flucs[2] |> normalize!, "--C1") 
    plot(padded_coarse_times, crit_flucs[crit_idx], "-k", label=row[:seed])
    plot(padded_coarse_times, crit_flucs[1], "--C0")
    plot(padded_coarse_times, crit_flucs[2], "--C1")     
    xlim(0., 1.)
    ylim(0, )
    xlabel("\$s - s_*\$")
    legend(frameon=false)

    tight_layout()
    # savefig(("test_plots/seed_" * plot_seed * ".pdf"))
end